In [41]:
pip install scikit-learn pandas numpy ucimlrepo joblib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

In [43]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 
  
# data (as pandas dataframes) 
X = heart_disease.data.features 
y = heart_disease.data.targets 
  
# metadata 
print(heart_disease.metadata) 
  
# variable information 
print(heart_disease.variables)


{'uci_id': 45, 'name': 'Heart Disease', 'repository_url': 'https://archive.ics.uci.edu/dataset/45/heart+disease', 'data_url': 'https://archive.ics.uci.edu/static/public/45/data.csv', 'abstract': '4 databases: Cleveland, Hungary, Switzerland, and the VA Long Beach', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 303, 'num_features': 13, 'feature_types': ['Categorical', 'Integer', 'Real'], 'demographics': ['Age', 'Sex'], 'target_col': ['num'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1989, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C52P4X', 'creators': ['Andras Janosi', 'William Steinbrunn', 'Matthias Pfisterer', 'Robert Detrano'], 'intro_paper': {'ID': 231, 'type': 'NATIVE', 'title': 'International application of a new probability algorithm for the diagnosis of coronary artery disease.', 'authors': 'R. Detrano, A. Jánosi, W. Steinbrunn, M

In [44]:
# Exploração inicial dos dados
print("Shape dos dados:")
print(f"X: {X.shape}")
print(f"y: {y.shape}")
print("\nPrimeiras linhas de X:")
print(X.head())
print("\nPrimeiras linhas de y:")
print(y.head())
print("\nInformações sobre os dados:")
print(X.info())
print("\nValores missing:")
print(X.isnull().sum())
print("\nDistribuição da variável target:")
print(y.value_counts())


Shape dos dados:
X: (303, 13)
y: (303, 1)

Primeiras linhas de X:
   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   1       145   233    1        2      150      0      2.3      3   
1   67    1   4       160   286    0        2      108      1      1.5      2   
2   67    1   4       120   229    0        2      129      1      2.6      2   
3   37    1   3       130   250    0        0      187      0      3.5      3   
4   41    0   2       130   204    0        2      172      0      1.4      1   

    ca  thal  
0  0.0   6.0  
1  3.0   3.0  
2  2.0   7.0  
3  0.0   3.0  
4  0.0   3.0  

Primeiras linhas de y:
   num
0    0
1    2
2    1
3    0
4    0

Informações sobre os dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 

## Limpeza e Preparação dos Dados

In [45]:
# Limpeza básica dos dados
# Criar cópia para trabalhar
df = X.copy()
target = y.copy()

# Verificar e tratar valores missing
print("Valores missing antes da limpeza:")
print(df.isnull().sum())

# Remover linhas com valores missing (estratégia simples)
# Alternativamente, poderia usar imputação
df_clean = df.dropna()
target_clean = target.loc[df_clean.index]

print(f"\nShape após limpeza: {df_clean.shape}")
print(f"Linhas removidas: {len(df) - len(df_clean)}")

# Verificar tipos de dados e converter se necessário
print("\nTipos de dados:")
print(df_clean.dtypes)


Valores missing antes da limpeza:
age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          4
thal        2
dtype: int64

Shape após limpeza: (297, 13)
Linhas removidas: 6

Tipos de dados:
age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca          float64
thal        float64
dtype: object


## Split Train/Test e Treinamento do Modelo

In [46]:


# Preparar target (assumindo que tem valores 0-4, converter para binário)
# 0 = sem doença, 1-4 = com doença
y_binary = (target_clean.values.ravel() > 0).astype(int)

print(f"Distribuição do target binário:")
print(f"Sem doença (0): {sum(y_binary == 0)}")
print(f"Com doença (1): {sum(y_binary == 1)}")

# Split train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    df_clean, y_binary, test_size=0.2, random_state=42, stratify=y_binary
)

print(f"\nTamanho do conjunto de treino: {X_train.shape}")
print(f"Tamanho do conjunto de teste: {X_test.shape}")


Distribuição do target binário:
Sem doença (0): 160
Com doença (1): 137

Tamanho do conjunto de treino: (237, 13)
Tamanho do conjunto de teste: (60, 13)


In [47]:
# Normalização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Dados normalizados com StandardScaler")


Dados normalizados com StandardScaler


In [48]:
# Treinar modelo Random Forest
print("Treinando Random Forest...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
rf_model.fit(X_train_scaled, y_train)

# Predições
y_pred_rf = rf_model.predict(X_test_scaled)
y_pred_proba_rf = rf_model.predict_proba(X_test_scaled)[:, 1]

# Métricas
accuracy_rf = accuracy_score(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(y_test, y_pred_proba_rf)

print(f"\n{'='*50}")
print("RANDOM FOREST - Resultados:")
print(f"{'='*50}")
print(f"Acurácia: {accuracy_rf:.4f}")
print(f"ROC AUC: {roc_auc_rf:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred_rf, target_names=['Sem Doença', 'Com Doença']))
print(f"\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))


Treinando Random Forest...

RANDOM FOREST - Resultados:
Acurácia: 0.8667
ROC AUC: 0.9420

Classification Report:
              precision    recall  f1-score   support

  Sem Doença       0.85      0.91      0.88        32
  Com Doença       0.88      0.82      0.85        28

    accuracy                           0.87        60
   macro avg       0.87      0.86      0.87        60
weighted avg       0.87      0.87      0.87        60


Confusion Matrix:
[[29  3]
 [ 5 23]]


In [49]:
# Treinar modelo Logistic Regression (para comparação)
print("Treinando Logistic Regression...")
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_scaled, y_train)

# Predições
y_pred_lr = lr_model.predict(X_test_scaled)
y_pred_proba_lr = lr_model.predict_proba(X_test_scaled)[:, 1]

# Métricas
accuracy_lr = accuracy_score(y_test, y_pred_lr)
roc_auc_lr = roc_auc_score(y_test, y_pred_proba_lr)

print(f"\n{'='*50}")
print("LOGISTIC REGRESSION - Resultados:")
print(f"{'='*50}")
print(f"Acurácia: {accuracy_lr:.4f}")
print(f"ROC AUC: {roc_auc_lr:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred_lr, target_names=['Sem Doença', 'Com Doença']))
print(f"\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_lr))


Treinando Logistic Regression...

LOGISTIC REGRESSION - Resultados:
Acurácia: 0.8333
ROC AUC: 0.9498

Classification Report:
              precision    recall  f1-score   support

  Sem Doença       0.82      0.88      0.85        32
  Com Doença       0.85      0.79      0.81        28

    accuracy                           0.83        60
   macro avg       0.83      0.83      0.83        60
weighted avg       0.83      0.83      0.83        60


Confusion Matrix:
[[28  4]
 [ 6 22]]


## Guardar Modelo e Scaler

In [50]:
# Selecionar o melhor modelo (baseado em ROC AUC)
if roc_auc_rf >= roc_auc_lr:
    best_model = rf_model
    best_model_name = "RandomForest"
    best_accuracy = accuracy_rf
    best_roc_auc = roc_auc_rf
else:
    best_model = lr_model
    best_model_name = "LogisticRegression"
    best_accuracy = accuracy_lr
    best_roc_auc = roc_auc_lr

print(f"Melhor modelo: {best_model_name}")
print(f"Acurácia: {best_accuracy:.4f}")
print(f"ROC AUC: {best_roc_auc:.4f}")

# Guardar modelo e scaler
joblib.dump(best_model, 'heart_disease_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

# Guardar também os nomes das features para referência
feature_names = df_clean.columns.tolist()
joblib.dump(feature_names, 'feature_names.pkl')

print(f"\nModelo guardado em: heart_disease_model.pkl")
print(f"Scaler guardado em: scaler.pkl")
print(f"Feature names guardados em: feature_names.pkl")


Melhor modelo: LogisticRegression
Acurácia: 0.8333
ROC AUC: 0.9498

Modelo guardado em: heart_disease_model.pkl
Scaler guardado em: scaler.pkl
Feature names guardados em: feature_names.pkl


In [51]:
import joblib
import numpy as np
import pandas as pd
from typing import Union, List, Dict

In [52]:
class HeartDiseasePredictor:
    """Classe para fazer predições de doença cardíaca"""
    
    def __init__(self, model_path='heart_disease_model.pkl', 
                 scaler_path='scaler.pkl',
                 feature_names_path='feature_names.pkl'):
        """
        Inicializa o preditor carregando o modelo e scaler
        
        Args:
            model_path: Caminho para o ficheiro do modelo
            scaler_path: Caminho para o ficheiro do scaler
            feature_names_path: Caminho para os nomes das features
        """
        self.model = joblib.load(model_path)
        self.scaler = joblib.load(scaler_path)
        self.feature_names = joblib.load(feature_names_path)
        
        print(f"Modelo carregado: {type(self.model).__name__}")
        print(f"Features esperadas ({len(self.feature_names)}): {self.feature_names}")
    
    def predict(self, features: Union[np.ndarray, pd.DataFrame, List, Dict]) -> Dict:
        """
        Faz predição sobre as features fornecidas
        
        Args:
            features: Features do paciente (array, DataFrame, lista ou dicionário)
        
        Returns:
            Dicionário com a predição e probabilidade
        """
        # Converter para DataFrame se necessário
        if isinstance(features, dict):
            features_df = pd.DataFrame([features])
        elif isinstance(features, list):
            features_df = pd.DataFrame([features], columns=self.feature_names)
        elif isinstance(features, np.ndarray):
            if features.ndim == 1:
                features = features.reshape(1, -1)
            features_df = pd.DataFrame(features, columns=self.feature_names)
        elif isinstance(features, pd.DataFrame):
            features_df = features
        else:
            raise ValueError("Formato de features não suportado")
        
        # Validar que temos todas as features necessárias
        if not all(col in features_df.columns for col in self.feature_names):
            missing = [col for col in self.feature_names if col not in features_df.columns]
            raise ValueError(f"Features em falta: {missing}")
        
        # Ordenar colunas na ordem correta
        features_df = features_df[self.feature_names]
        
        # Normalizar
        features_scaled = self.scaler.transform(features_df)
        
        # Predição
        prediction = self.model.predict(features_scaled)[0]
        probability = self.model.predict_proba(features_scaled)[0]
        
        # Resultado
        result = {
            'prediction': int(prediction),
            'resultado': 'TEM doença cardíaca' if prediction == 1 else 'NÃO TEM doença cardíaca',
            'probabilidade_sem_doenca': float(probability[0]),
            'probabilidade_com_doenca': float(probability[1]),
            'confianca': float(max(probability))
        }
        
        return result
    
    def predict_batch(self, features_list: Union[pd.DataFrame, List[Dict]]) -> List[Dict]:
        """
        Faz predições em lote
        
        Args:
            features_list: Lista de features ou DataFrame
        
        Returns:
            Lista de dicionários com predições
        """
        if isinstance(features_list, pd.DataFrame):
            return [self.predict(row.to_dict()) for _, row in features_list.iterrows()]
        else:
            return [self.predict(features) for features in features_list]

## Exemplos de Uso do Preditor

In [53]:
# Carregar o preditor
predictor = HeartDiseasePredictor()

print("\n" + "="*70)
print("EXEMPLO DE USO DO PREDITOR")
print("="*70)

Modelo carregado: LogisticRegression
Features esperadas (13): ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']

EXEMPLO DE USO DO PREDITOR


In [54]:
# Exemplo 1: Predição com dicionário
print("\n1. Predição com dicionário:")
paciente1 = {
    'age': 63,
    'sex': 1,
    'cp': 3,
    'trestbps': 145,
    'chol': 233,
    'fbs': 1,
    'restecg': 0,
    'thalach': 150,
    'exang': 0,
    'oldpeak': 2.3,
    'slope': 0,
    'ca': 0,
    'thal': 1
}

resultado1 = predictor.predict(paciente1)
print(f"Resultado: {resultado1['resultado']}")
print(f"Probabilidade de doença: {resultado1['probabilidade_com_doenca']:.2%}")
print(f"Confiança: {resultado1['confianca']:.2%}")


1. Predição com dicionário:
Resultado: NÃO TEM doença cardíaca
Probabilidade de doença: 2.75%
Confiança: 97.25%


In [55]:
# Exemplo 2: Predição com lista (na ordem das features)
print("\n2. Predição com lista:")
paciente2 = [67, 1, 0, 160, 286, 0, 0, 108, 1, 1.5, 1, 3, 2]

resultado2 = predictor.predict(paciente2)
print(f"Resultado: {resultado2['resultado']}")
print(f"Probabilidade de doença: {resultado2['probabilidade_com_doenca']:.2%}")
print(f"Confiança: {resultado2['confianca']:.2%}")


2. Predição com lista:
Resultado: TEM doença cardíaca
Probabilidade de doença: 77.52%
Confiança: 77.52%


In [56]:
# Exemplo 3: Predição em lote (múltiplos pacientes)
print("\n3. Predição em lote (múltiplos pacientes):")
pacientes = [
    {'age': 54, 'sex': 1, 'cp': 0, 'trestbps': 140, 'chol': 239, 'fbs': 0, 
     'restecg': 1, 'thalach': 160, 'exang': 0, 'oldpeak': 1.2, 'slope': 0, 'ca': 0, 'thal': 2},
    {'age': 41, 'sex': 0, 'cp': 1, 'trestbps': 130, 'chol': 204, 'fbs': 0, 
     'restecg': 0, 'thalach': 172, 'exang': 0, 'oldpeak': 1.4, 'slope': 2, 'ca': 0, 'thal': 2},
]

resultados = predictor.predict_batch(pacientes)
for i, res in enumerate(resultados, 1):
    print(f"\nPaciente {i}: {res['resultado']} (Prob: {res['probabilidade_com_doenca']:.2%})")


3. Predição em lote (múltiplos pacientes):

Paciente 1: NÃO TEM doença cardíaca (Prob: 1.54%)

Paciente 2: NÃO TEM doença cardíaca (Prob: 1.51%)


In [57]:
print("\n" + "="*70)
print("Para usar este script:")
print("1. Importe a classe: from predict import HeartDiseasePredictor")
print("2. Crie uma instância: predictor = HeartDiseasePredictor()")
print("3. Faça predições: resultado = predictor.predict(features)")
print("="*70)


Para usar este script:
1. Importe a classe: from predict import HeartDiseasePredictor
2. Crie uma instância: predictor = HeartDiseasePredictor()
3. Faça predições: resultado = predictor.predict(features)
